In [51]:
import cv2,sys, copy, scipy
import torch
import fiona, rasterio
import numpy as np
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision 
import torchvision.models as models
import albumentations as A

from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader,Dataset
from shapely import geometry
from rasterio.mask import mask 
from tqdm import tqdm
from datetime import datetime
from extractor_helper import extractor
from torch.utils.tensorboard import SummaryWriter
from IPython.display import clear_output


## for reprodusability
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

ModuleNotFoundError: No module named 'albumentations'

## Data preprosseing 

In [48]:
class mydata(Dataset):
    def __init__(self, data, mode='train'):
        self.data = data
        self.mode = model
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = torch.tensor(self.data[idx][0]).permute(2,0,1).float()
        y = torch.tensor(self.data[idx][1])
        if self.model == 'train':
        return X, y

IndentationError: expected an indented block (<ipython-input-48-2809dfca001b>, line 13)

In [18]:
# INTER_NEAREST – a nearest-neighbor interpolation.
# INTER_LINEAR – a bilinear interpolation (used by default)
# INTER_AREA – resampling using pixel area relation. ...
# INTER_CUBIC – a bicubic interpolation over 4×4 pixel neighborhood.
# INTER_LANCZOS4 – a Lanczos interpolation over 8×8 pixel neighborhood.

def processing(patchsize = 3, interporlation = cv2.INTER_LANCZOS4, size = (224,224)):
    
    test2_img = rasterio.open("data/test2/pp_2_sat_modified.tif")
    test2_points = fiona.open("data/test2/points_2_modified_Copy.shp", "r")
    test3_img = rasterio.open("data/test3/pp_3_sat_modified.tif")
    test3_points = fiona.open("data/test3/targets_Copy.shp", "r")
    test4_img = rasterio.open("data/test4/pp_4_sat_modified_spline.tif")
    test4_points = fiona.open("data/test4/modified_points_Copy.shp", "r")
    
    patch2, coordinates2, labels2 = extractor(test2_img,test2_points, size=patchsize, normalize=True, labeling=True)
    patch3, coordinates3, labels3 = extractor(test3_img,test3_points, size=patchsize, normalize=True, labeling=True)
    patch4, coordinates4, labels4 = extractor(test4_img,test4_points, size=patchsize, normalize=True, labeling=True)
    
    patch2 = [[cv2.resize(i,size,interpolation=interporlation), l] for i,l in zip(patch2, labels2)]
    patch3 = [[cv2.resize(i,size,interpolation=interporlation), l] for i,l in zip(patch3, labels3)]
    patch4 = [[cv2.resize(i,size,interpolation=interporlation), l] for i,l in zip(patch4, labels4)]
    data = list(patch2 + patch3 + patch4)
    
    zero = [i for i in data if i[1] == 0]
    one = [i for i in data if i[1] == 1.0]
    two = [i for i in data if i[1] == 2.0]
    three = [i for i in data if i[1] == 3.0]
    
    data =  two + zero[:48//3] + one[:48//3] + three[:48//3]
    
    for j, i in enumerate(data):
        if i[1] == 2.0:
            data[j][1] = 0
        else:
            data[j][1] = 1
            
    return np.array(data)

In [19]:
def loaders(data, valsize=150, bsize=30):
    
    val_idx = list(np.random.randint(0,len(data), valsize))
    train_idx = list(set(list(range(len(data)))) - set(val_idx))
    trainloader = DataLoader(mydata(data[train_idx]), batch_size=bsize,
                             shuffle=True, num_workers=8, pin_memory=True )
    testloader  = DataLoader(mydata(data[val_idx]), batch_size=bsize,
                             num_workers=8, pin_memory=True )
    
    return (train_idx, val_idx), trainloader, testloader

In [20]:
def train(model, epoches, criterion, optimizer, trainloader, testloader, device, exp_name=None, scheduler=None):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    if exp_name:
        writer = SummaryWriter('./logs_weeklearns/{}'.format(exp_name))
    else:
        writer = SummaryWriter('./logs_weeklearns/{}'.format('single_ep'))
        
    bestval = {'bestvalacc': 0 , 'epoch': None, 'trainacc@Bval': None, 'iter':0}
    for e in (range(1, epoches+1)):
        
        model.train()
        meanloss = {'counter':0, 'correct':0, 'total':0}
        for X, y in (trainloader):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            bestval['iter'] +=1
            meanloss['total'] += len(y)
            meanloss['correct'] += (torch.argmax(pred, dim=1) == y).sum().item()
            writer.add_scalar("train_loss", loss.item(), global_step=bestval['iter'])
            
        writer.add_scalar("train_epoch_Accuracy", meanloss['correct']/len(trainloader.dataset), global_step=e)
        print('Epoch:{} |train_accuracy:{}'.format(e, meanloss['correct']/meanloss['total']))
        model.eval()
        meanlossval = {'loss':0, 'counter':0, 'correct':0,'total':0}
        for X, y in testloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            meanlossval['loss'] += criterion(pred, y).item()
            meanlossval['total'] += len(y)
            meanlossval['correct'] += (torch.argmax(pred, dim=1) == y).sum().item()
            
        writer.add_scalar("test_epoch_Accuracy", meanlossval['correct']/len(testloader.dataset), global_step=e)
        print('Epoch:{} |test_accuracy:{}'.format(e, meanlossval['correct']/meanlossval['total']))
        print('--'*50)
        if scheduler:
            scheduler.step()
            
        if meanlossval['correct']/meanlossval['total'] > bestval['bestvalacc']:
            bestval['bestvalacc'] = meanlossval['correct']/meanlossval['total']
            bestval['trainacc@Bval'] = meanloss['correct']/meanloss['total']
            bestval['epoch'] = e
            best_model_wts = copy.deepcopy(model.state_dict())
            
    print(bestval)
    model.load_state_dict(best_model_wts)
    return model

In [24]:
def plot():
    plt.figure(figsize=(12,5))
    plt.subplot(1,3,1)
    plt.imshow(data[3][0][...,[2,3,5]]*10)
    plt.title('rgb view of a tree')
    plt.subplot(1,3,2)
    plt.hist([i[1] for i in data[split_idx[0]]], bins=5);
    plt.title('class distribution: train')
    plt.subplot(1,3,3)
    plt.hist([i[1] for i in data[split_idx[1]]], bins=5);
    plt.title('class distribution: test');
    plt.tight_layout()
    plt.show()

In [27]:
# plot()

In [44]:
patchsize = 3 ### size of a patch 
interporlation = cv2.INTER_LANCZOS4 ### interpolation for resizing image 

data = processing(patchsize=patchsize, 
                  interporlation=interporlation,)
batchsize = 10
valsize = int(len(data) * .15) ## validation size 15 percent

split_idx, trainloader, testloader = loaders(data, valsize=valsize, bsize=batchsize)

/home/dsdr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [45]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

####### SELECT MODELS and freeze weights
model = models.resnet18(pretrained=True)
model.requires_grad_(False);

###### CHANGE Classifier 
model.conv1 = nn.Conv2d(8,64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(512,2)
                         

model.to(device)
epoches = 15
learning_rate = 3e-3
w_decay = 5e-4

parameters  = [{'params': model.conv1.parameters(),'lr': 1e-4},
               {'params': model.fc.parameters()}]

# class_weights = torch.tensor(class_weights, dtype = torch.float32).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( parameters,
                              lr=learning_rate,
                              weight_decay=w_decay)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [7,10,13], gamma=0.1)
scheduler = None

params = {'lr':learning_rate,
          'epoches': epoches,
          'weight_dec': w_decay,
          'batch_size': batchsize,
          'interporlation': interporlation,
          'patch_size': patchsize,
          'scheduler': scheduler}

exp_name = datetime.now().isoformat() + str(params)
print(exp_name)

2020-08-15T23:27:36.598813{'lr': 0.003, 'epoches': 15, 'weight_dec': 0.0005, 'batch_size': 10, 'interporlation': 4, 'patch_size': 3, 'scheduler': None}


In [46]:
best_model = train(model, epoches, criterion, optimizer, trainloader, testloader, device, exp_name, scheduler )

Epoch:1 |train_accuracy:0.44871794871794873
Epoch:1 |test_accuracy:0.6153846153846154
----------------------------------------------------------------------------------------------------
Epoch:2 |train_accuracy:0.6025641025641025
Epoch:2 |test_accuracy:0.7692307692307693
----------------------------------------------------------------------------------------------------
Epoch:3 |train_accuracy:0.717948717948718
Epoch:3 |test_accuracy:0.6153846153846154
----------------------------------------------------------------------------------------------------
Epoch:4 |train_accuracy:0.7564102564102564
Epoch:4 |test_accuracy:0.6153846153846154
----------------------------------------------------------------------------------------------------
Epoch:5 |train_accuracy:0.8076923076923077
Epoch:5 |test_accuracy:0.6923076923076923
----------------------------------------------------------------------------------------------------
Epoch:6 |train_accuracy:0.7307692307692307
Epoch:6 |test_accuracy:0.61

In [70]:
with torch.no_grad():
    y_pred,y_true = [],[]
    for b, y in testloader:
        b = b.to(device)
        y_pred.extend(torch.argmax(best_model(b), dim=1).cpu().numpy())
        y_true.extend(y.numpy())

In [71]:
confusion_matrix(y_true, y_pred)

array([[65,  9,  3],
       [ 7, 34,  6],
       [ 1,  0, 20]])

In [26]:
# torch.save(best_model.state_dict(),'./best_ensumble/resnet50_82.pth')

In [47]:
%load_ext tensorboard

import os
logs_base_dir = "./logs_weeklearns/"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 20186), started 0:03:39 ago. (Use '!kill 20186' to kill it.)

### Fine Tuning 

In [73]:
for param in best_model.parameters():
    param.requires_grad = True

In [74]:
trainable_parameters = best_model.parameters()

learning_rate = 0
optimizer = torch.optim.Adam(
    trainable_parameters, lr=learning_rate, weight_decay=1e-4)

In [75]:
best_model = train(best_model, 1, criterion, optimizer, trainloader, testloader, device)

Epoch:1 |train_accuracy:0.828125
Epoch:1 |test_accuracy:0.8137931034482758
----------------------------------------------------------------------------------------------------
{'bestvalacc': 0.8137931034482758, 'epoch': 1, 'trainacc@Bval': 0.828125, 'iter': 28}


In [76]:
def set_learning_rate(optimizer, new_learning_rate):
    """Set learning rates of the optimizer to `new_learning_rate`."""
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_learning_rate

In [77]:
learning_rate = 1e-4
set_learning_rate(optimizer, learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,6 ,gamma=0.1,)

In [78]:
best_model = train(best_model, 20, criterion, optimizer, trainloader, testloader, device, scheduler=scheduler)

Epoch:1 |train_accuracy:0.8221153846153846
Epoch:1 |test_accuracy:0.46206896551724136
----------------------------------------------------------------------------------------------------
Epoch:2 |train_accuracy:0.8629807692307693
Epoch:2 |test_accuracy:0.7517241379310344
----------------------------------------------------------------------------------------------------
Epoch:3 |train_accuracy:0.875
Epoch:3 |test_accuracy:0.6620689655172414
----------------------------------------------------------------------------------------------------
Epoch:4 |train_accuracy:0.9026442307692307
Epoch:4 |test_accuracy:0.6896551724137931
----------------------------------------------------------------------------------------------------
Epoch:5 |train_accuracy:0.9302884615384616
Epoch:5 |test_accuracy:0.7517241379310344
----------------------------------------------------------------------------------------------------
Epoch:6 |train_accuracy:0.9471153846153846
Epoch:6 |test_accuracy:0.77931034482758

In [78]:
# torch.save(best_model.state_dict(),'./best_ensumble/wide_resnet50_80.pth')